# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.121859e+02     1.478726e+00
 * time: 0.12488985061645508
     1     1.091177e+01     1.067664e+00
 * time: 1.0175549983978271
     2    -1.230745e+01     1.043913e+00
 * time: 1.158998966217041
     3    -3.425900e+01     9.032160e-01
 * time: 1.3135309219360352
     4    -4.789073e+01     7.520529e-01
 * time: 1.4609730243682861
     5    -5.705595e+01     3.074327e-01
 * time: 1.6020610332489014
     6    -5.978100e+01     2.880649e-01
 * time: 1.7172129154205322
     7    -6.091281e+01     1.392807e-01
 * time: 1.8342208862304688
     8    -6.141858e+01     4.118639e-02
 * time: 1.9427218437194824
     9    -6.166344e+01     3.724459e-02
 * time: 2.059459924697876
    10    -6.182938e+01     4.932860e-02
 * time: 2.170297861099243
    11    -6.196563e+01     2.723432e-02
 * time: 2.2856569290161133
    12    -6.205699e+01     1.991604e-02
 * time: 2.4043428897857666
    13    -6.211610e+01     1.536877e-02
 * time: 2.516311883926

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671061
    AtomicLocal         -18.8557617
    AtomicNonlocal      14.8522602
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485356 
    Xc                  -19.3336814

    total               -62.261666460690
